In [1]:

import os
import random
import json
import paddle
import sys
import numpy as np
import paddle.fluid as fluid
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
from paddle.fluid.param_attr import ParamAttr
import copy
import cv2
from PIL import Image

In [2]:
# 图像增广

def rotate(image, angle, center=None, scale=1.0):
    (h, w) = image.shape[:2]
    if center is None:
        center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

#翻转

def flip(image, orentation):
    img_new = cv2.flip(img,orentation)
    return img_new

#移动

def move(img, direction, stride):
    if direction == 'left':        
        M = np.float32([[1, 0, -1*stride], [0, 1, 0]])
    elif direction == 'right':
        M = np.float32([[1, 0, stride], [0, 1, 0]])
    elif direction == 'up':
        M = np.float32([[1, 0, 0], [0, 1, -1 * stride]])
    elif direction == 'down':
        M = np.float32([[1, 0, 0], [0, 1, stride]])
    shifted = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return shifted

#大小缩放

def resize(img, Wscale, Hscale):
    img_new = cv2.resize(img, (int(Wscale * img.shape[0]), int(Hscale * img.shape[1])))
    return img_new

In [3]:
img_dir1 = './data/maskDetect/maskimages'
new_dir = './data/aug_data'
if not os.path.exists(new_dir) :
    os.mkdir(new_dir)
if not os.path.exists(new_dir + '/maskimages/') :
    os.mkdir(new_dir + '/maskimages/')
for i in os.listdir(img_dir1):
    img = cv2.imread(img_dir1 + '/' + i)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '.jpg', img)
    img_new1 = rotate(img,90)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_rot90.jpg', img_new1)
    img_new2 = rotate(img,270)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_rot270.jpg', img_new2)
    img_new3 = flip(img,0)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_flip0.jpg', img_new3)
    img_new4 = flip(img,1)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_flip1.jpg', img_new4)
    img_new5 = flip(img,-1)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_flip-1.jpg', img_new5)
    img_new6 = move(img,'left',100)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_left.jpg', img_new6)
    img_new7 = move(img,'right',100)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_right.jpg', img_new7)
    img_new8 = move(img,'up',100)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_up.jpg', img_new8)
    img_new9 = move(img,'down',100)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_down.jpg', img_new9)
    img_new10 = resize(img, 0.8, 0.8)
    cv2.imwrite(new_dir + '/maskimages/' + i[0:7] + '_resize.jpg', img_new10)

In [4]:
img_dir2 = './data/maskDetect/nomaskimages'
new_dir = './data/aug_data'
if not os.path.exists(new_dir) :
    os.mkdir(new_dir)
if not os.path.exists(new_dir + '/nomaskimages/') :
    os.mkdir(new_dir +'/nomaskimages/')
for i in os.listdir(img_dir2):
    img = cv2.imread(img_dir2 + '/' + i)
    if img is None:
        continue
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '.jpg', img)
    img_new1 = rotate(img,90)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_rot90.jpg', img_new1)
    img_new2 = rotate(img,270)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_rot270.jpg', img_new2)
    img_new3 = flip(img,0)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_flip0.jpg', img_new3)
    img_new4 = flip(img,1)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_flip1.jpg', img_new4)
    img_new5 = flip(img,-1)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_flip-1.jpg', img_new5)
    img_new6 = move(img,'left',100)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_left.jpg', img_new6)
    img_new7 = move(img,'right',100)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_right.jpg', img_new7)
    img_new8 = move(img,'up',100)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_up.jpg', img_new8)
    img_new9 = move(img,'down',100)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_down.jpg', img_new9)
    img_new10 = resize(img, 0.8, 0.8)
    cv2.imwrite(new_dir + '/nomaskimages/' + i[0:7] + '_resize.jpg', img_new10)

In [5]:
def get_data_list(train_list_path,eval_list_path):
    label_dict = {}
    #存放所有类别的信息
    class_detail = []
    #获取所有类别保存的文件夹名称
    data_list_path = './data/aug_data/'
    class_dirs = os.listdir(data_list_path)  
    #总的图像数量
    all_class_images = 0
    #存放类别标签
    class_label = 0
    #存放类别数目
    class_dim = 0
    #存储要写进eval.txt和train.txt中的内容
    trainer_list = []
    eval_list = []
    #读取每个类别，['maskimages', 'nomaskimages']
    for class_dir in class_dirs:
        if class_dir != ".ipynb_checkpoints":
#         if class_dir != ".DS_Store" or ".ipynb_checkpoints":
            class_dim += 1
            #每个类别的信息
            class_detail_list = {}
            eval_sum = 0
            trainer_sum = 0
            #统计每个类别有多少张图片
            class_sum = 0
            #获取各类别路径 
            path = data_list_path + class_dir
            # 获取该类下所有图片
            img_paths = os.listdir(path)
            # 遍历文件夹下的每个图片
            for img_path in img_paths:                                  
                name_path = path + '/' + img_path                       # 每张图片的路径
                if class_sum % 10 == 0:                                 # 每10张图片取一个做验证数据
                    eval_sum += 1                                       # test_sum为测试数据的数目
                    eval_list.append(name_path + "\t%d" % class_label + "\n")
                else:
                    trainer_sum += 1 
                    trainer_list.append(name_path + "\t%d" % class_label + "\n")#trainer_sum测试数据的数目
                class_sum += 1                                          #每类图片的数目
                all_class_images += 1                                   #所有类图片的数目

            
            # 说明的json文件的class_detail数据
            class_detail_list['class_name'] = class_dir             #类别名称，如jiangwen
            class_detail_list['class_label'] = class_label          #类别标签
            class_detail_list['class_eval_images'] = eval_sum       #该类数据的测试集数目
            class_detail_list['class_trainer_images'] = trainer_sum #该类数据的训练集数目
            class_detail.append(class_detail_list)  

            #初始化标签列表
            label_dict[str(class_label)] = class_dir
            class_label += 1 

    #乱序  
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image) 
            
    random.shuffle(trainer_list)
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image) 

    # 说明的json文件信息
    readjson = {}
    readjson['all_class_name'] = data_list_path                  #文件父目录
    readjson['all_class_images'] = all_class_images
    readjson['class_detail'] = class_detail
    jsons = json.dumps(readjson, sort_keys = True, indent = 4, separators = (',', ': '))
    with open('./data/readme.json','w') as f:
        f.write(jsons)
    print ('生成数据列表完成！')

In [6]:
def custom_reader(file_list):
    def reader():
        with open(file_list, 'r') as f:
            lines = [line.strip() for line in f]
            for line in lines:
                img_path, lab = line.strip().split('\t')
                #读取图像
                #img = cv2.imread(img)
                img = cv2.imread(img_path)
                #转换尺寸为（100，100）
                img = cv2.resize(img, (224, 224))
                #转换数据类型为float32
                img = np.array(img).astype('float32')
                #修改数据形状为paddle默认的（通道，高度，宽度）
                img = img.transpose((2, 0, 1))
                #对图像数据进行归一化
                img = img/255.0 
                yield img, int(lab) 
    return reader



In [7]:
train_list_path = './data/train.txt'
eval_list_path = './data/eval.txt'
batch_size = 32

#删除文件里内容
with open(train_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(eval_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
    
#生成数据列表   
get_data_list(train_list_path,eval_list_path)

'''
构造数据提供器
'''
train_reader = paddle.batch(custom_reader(train_list_path),
                            batch_size = batch_size,
                            drop_last = True) 
#drop_last若设置为True，则当最后一个batch不等于batch_size时，丢弃最后一个batch；若设置为False，则不会。默认值为False
eval_reader = paddle.batch(custom_reader(eval_list_path),
                            batch_size = batch_size,
                            drop_last = True)

生成数据列表完成！


In [8]:

def  VGG16(inputs):
    def conv_layer(inputs, num_filters, name = None, filter_size = 3, stride = 1, act = 'relu'):
        conv = fluid.layers.conv2d(input = inputs,
                                   num_filters = num_filters,
                                   filter_size = filter_size,
                                   stride = stride,
                                   padding = (filter_size - 1) // 2,
                                   act = act,
                                   param_attr = ParamAttr(name=name + "_weights"))  # 参数属性设置
        return conv
    depth = [2, 2, 3, 3, 3]
    num_filters = [64, 128, 256, 512, 512]
    out = inputs
    for i in range(len(depth)):
        for j in range(depth[i]):
            conv_name = 'conv' + str(i+1) + '_' + str(j+1)
            out = conv_layer(out, num_filters[i], name = conv_name)
        out = fluid.layers.pool2d(out, pool_size=2, pool_stride=2, pool_type='max')
    return out

In [9]:

image = fluid.layers.data(name='image', shape=[3,224,224], dtype='float32')
label = fluid.layers.data(name='label', shape=[1], dtype='int64')

In [10]:

# 获取分类器的上一层
pool = VGG16(image)
# 停止梯度下降
pool.stop_gradient = True
# 由这里创建一个基本的主程序
base_model_program = fluid.default_main_program().clone()

# 这里再重新加载网络的分类器，大小为本项目的分类大小
out = fluid.layers.fc(input=pool, size=2, act='softmax')


In [11]:

# 获取损失函数和准确率函数
cost = fluid.layers.cross_entropy(input=out, label=label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input=out, label=label)

# 定义优化方法
optimizer = fluid.optimizer.AdamOptimizer(learning_rate=1e-3)
opts = optimizer.minimize(avg_cost)


# 定义训练场所
place = fluid.CUDAPlace(0)#用GPU训练
#place = fluid.CPUPlace() #用CPU训练
exe = fluid.Executor(place)

# 进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [12]:
src_pretrain_model_path = './VGG16_pretrained/'
def if_exist(var):
    path = os.path.join(src_pretrain_model_path, var.name)
    exist = os.path.exists(path)
    return exist
fluid.io.load_vars(executor = exe, dirname = src_pretrain_model_path, 
                   predicate = if_exist, main_program = base_model_program)

In [ ]:
# DataFeeder 负责将reader(读取器)返回的数据转成一种特殊的数据结构，使它们可以输入到 Executor 和 ParallelExecutor 中
feeder = fluid.DataFeeder(place = place, feed_list = [image, label])
epoch = 3

# 训练10次
for pass_id in range(epoch):
    # 进行训练
    for batch_id, data in enumerate(train_reader()):
        train_cost, train_acc, pred = exe.run(program = fluid.default_main_program(),
                                        feed = feeder.feed(data),
                                        fetch_list = [avg_cost, acc, out])

        # 每100个batch打印一次信息
        if batch_id % 100 == 0:
            print('Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f' %
                  (pass_id, batch_id, train_cost[0], train_acc[0]))
        if train_acc > 0.95:
            break

In [ ]:
fluid.io.save_inference_model(dirname = './model/',feeded_var_names = ['image'],
                                              target_vars = [out],
                                              main_program = fluid.default_main_program(),
                                              executor = exe)

In [ ]:
test_program = fluid.default_main_program().clone(for_test = True)

In [ ]:

for _,data in enumerate(eval_reader()):
    test_cost, test_acc = exe.run(program = test_program,
                                        feed = feeder.feed(data),
                                        fetch_list = [avg_cost, acc])
    print('Cost:%0.5f, Accuracy:%0.5f' %(test_cost[0], test_acc[0]))

In [ ]:
def load_image(img_path):
    '''
    预测图片预处理
    '''
    img = Image.open(img_path) 
    if img.mode != 'RGB': 
        img = img.convert('RGB') 
    img = img.resize((224, 224), Image.BILINEAR)
    img = np.array(img).astype('float32') 
    img = img.transpose((2, 0, 1))  # HWC to CHW 
    img = img/255                # 像素值归一化 
    img = img[np.newaxis,:]
    return img
img1 = load_image('./test.jpg')
img2 = load_image('./text.jpg')


In [ ]:
[inference_program, feed_target_names, fetch_targets] = fluid.io.load_inference_model(dirname='./model/', executor=exe)

In [ ]:

label = exe.run(inference_program, feed={'image':img2}, fetch_list=fetch_targets)
#display(Image.open('./text.jpg'))
plt.imshow(Image.open('./text.jpg'))
print('识别结果为：',np.argmax(label))